# 【模型】Logistics疫情预测模型

数据路径：`/home/mw/input/COVID8877/covid/district/`



![Image Name](https://cdn.kesci.com/upload/image/rczk3m5adu.png)


## 导入模块

In [1]:
import pandas as pd
import matplotlib.dates as mdates
from matplotlib.pyplot import MultipleLocator
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

path = '/opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/SourceHanSansSC-Regular.ttf' # 字体路径
prop = font_manager.FontProperties(fname=path)  # 加载字体
plt.rcParams['font.family'] = prop.get_name()   # 显示中文
plt.rcParams['axes.unicode_minus'] = False      # 显示负号

## 数据预处理

In [2]:
date_list = pd.date_range(start='2022-02-27', end='2022-04-18')
date_str_list = [str(i.year)+str(i.month).zfill(2)+str(i.day).zfill(2) for i in date_list]

path = '/home/mw/input/COVID8877/covid/district/'
df_list = []
for date in date_str_list:
    df = pd.read_excel(path + date + '_city.xlsx')
    df_list.append(df)
df_city = pd.concat(df_list)
df_city['累计总数'] = df_city['总数'].cumsum()
df_city = df_city.reset_index(drop=True)
df_city

,时间,总数,确诊,无症状,无症状转归,死亡,累计总数
0,2022-02-27,1,0,1,NaN,NaN,1
1,2022-02-28,3,0,3,NaN,NaN,4
2,2022-03-01,2,1,1,NaN,NaN,6
3,2022-03-02,0,0,0,NaN,NaN,6
4,2022-03-03,16,2,14,NaN,NaN,22
5,2022-03-04,19,3,16,NaN,NaN,41
6,2022-03-05,28,0,28,NaN,NaN,69
7,2022-03-06,48,3,45,NaN,NaN,117
8,2022-03-08,55,4,51,NaN,NaN,172
9,2022-03-08,65,3,62,NaN,NaN,237


## 数据可视化

### 每日新增病例数

In [3]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
plt.tick_params(labelsize=15)
ax.plot_date(df_city['时间'].iloc[0:15], df_city['总数'].iloc[0:15], 'o-')
plt.ylabel('每日新增病例数',fontsize=15)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d\n%Y'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=2))
plt.show()

<Figure size 720x432 with 1 Axes>

In [4]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
plt.tick_params(labelsize=15)
ax.plot_date(df_city['时间'], df_city['总数'], 'o-')
plt.ylabel('每日新增病例数',fontsize=15)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d\n%Y'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=4))
plt.show()

<Figure size 720x432 with 1 Axes>

### 累计确诊病例数

In [5]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
plt.tick_params(labelsize=15)
ax.plot_date(df_city['时间'], df_city['累计总数'], 'o-')
plt.ylabel('累计确诊病例数',fontsize=15)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d\n%Y'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=4))

plt.show()

<Figure size 720x432 with 1 Axes>

## Logistics模型拟合

In [6]:
def logistic_increase_function(t, P0):
    # logistic生长函数：t:time   P0:initial_value    K:capacity  r:increase_rate
    # 后面将对r和K进行网格优化
    r = hyperparameters_r
    K = hyperparameters_K
    exp_value = np.exp(r * (t))
    return (K * exp_value * P0) / (K + (exp_value - 1) * P0)

In [7]:
y_data = df_city['累计总数']#.iloc[0:44]
x_data = np.asarray(range(0,len(y_data)))

In [8]:
k_range = np.arange(10000, 602000, 2000)
r_range = np.arange(0, 1, 0.01)
popt = None
mse = float("inf")
i = 0
# 网格搜索来优化r和K参数
r = None
k = None
for k_ in k_range:
    global hyperparameters_K
    hyperparameters_K = k_
    for r_ in r_range:
        global hyperparameters_r
        hyperparameters_r = r_
        # 用非线性最小二乘法拟合
        popt_, pcov_ = curve_fit(logistic_increase_function, x_data, y_data)
        # 采用均方误准则选择最优参数
        mse_ = mean_squared_error(y_data, logistic_increase_function(x_data, *popt_))
        if mse_ <= mse:
            mse = mse_
            popt = popt_
            r = r_
            k = k_
        i = i+1
        print('\r当前进度：{0}{1}%'.format('▉'*int(i*10/len(k_range)/len(r_range)),int(i*100/len(k_range)/len(r_range))), end='')

当前进度：▉▉▉▉▉▉▉▉▉▉100%

In [9]:
hyperparameters_K = k
hyperparameters_r = r
popt, pcov = curve_fit(logistic_increase_function, x_data, y_data)
print("K:capacity  P0:initial_value   r:increase_rate")
print(hyperparameters_K, popt, hyperparameters_r)

K:capacity  P0:initial_value   r:increase_rate
492000 [33.51917937] 0.22


## Logistics模型预测

In [10]:
future = np.linspace(0, 60, 61)
future = np.array(future)
future_predict = logistic_increase_function(future, popt)
diff = np.diff(future_predict)
diff = np.insert(diff, 0, np.nan)
future_predict = logistic_increase_function(future, 27)

### 预测累计确诊病例数

In [11]:
plt.figure(figsize=(10, 4))
plt.tick_params(labelsize=15)
plt.scatter(x_data, y_data, s=35, marker='.', c = "dimgray", label="确诊人数")
plt.plot(future, future_predict, 'r', linewidth=1.5, label='预测曲线')
plt.ylabel('累计确诊病例数',fontsize=15)
plt.legend(frameon=False,fontsize=15)
plt.show()

<Figure size 720x288 with 1 Axes>

### 预测每日新增病例数

In [12]:
# 最佳估计
future_predict = logistic_increase_function(future, 27)
diff = np.diff(future_predict)
diff = np.insert(diff, 0, np.nan)
# 悲观估计
future_predict1 = logistic_increase_function(future, 20)
diff1 = np.diff(future_predict1)
diff1 = np.insert(diff1, 0, np.nan)
# 乐观估计
future_predict2 = logistic_increase_function(future, 34)
diff2 = np.diff(future_predict2)
diff2 = np.insert(diff2, 0, np.nan)

# 日期列表
date_list1 = pd.date_range(start='2022-02-27', end='2022-04-18')
date_list2 = pd.date_range(start='2022-02-27', end='2022-04-28')

# 绘图
fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(111)
ax.tick_params(labelsize=15)
'''
plt.plot(np.asarray(range(0,len(df_city['总数']))), df_city['总数'], 'o-', label='观测数据')
plt.plot(future, diff, "orange", linewidth=1.5, label='最佳估计')
plt.plot(future, diff1, "red", linewidth=1.5, label='悲观估计')
plt.plot(future, diff2, "green", linewidth=1.5, label='乐观估计')
'''
ax.plot_date(df_city['时间'], df_city['总数'], 'o-', label='观测数据')
ax.plot_date(date_list2, diff, "orange", linewidth=1.5, label='最佳估计')
ax.plot_date(date_list2, diff1, "red", linewidth=1.5, label='悲观估计')
ax.plot_date(date_list2, diff2, "green", linewidth=1.5, label='乐观估计')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d\n%Y'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=5))

plt.legend(frameon=False, fontsize=15)
plt.ylabel('每日新增确诊病例数',fontsize=15)
plt.show()

<Figure size 720x288 with 1 Axes>

In [13]:
print('4月20日疫情新增区间为{}-{}，最优估计为{}'.format(int(diff2[52]),int(diff1[52]),int(diff[52])))
print('4月21日疫情新增区间为{}-{}，最优估计为{}'.format(int(diff2[53]),int(diff1[53]),int(diff[53])))

4月20日疫情新增区间为13657-19050，最优估计为15946
4月21日疫情新增区间为11632-16768，最优估计为13758
